<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/NER/gemini_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-aiplatform
!pip install -q -U google-generativeai
!pip install -q groq
!pip install -U accelerate bitsandbytes datasets evaluate
!pip install -U peft transformers trl
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
import time
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusol

# **GEMINI**

In [ ]:
# For Google Colab settings
from google.colab import userdata, drive

# This will prompt for authorization
drive.mount('/content/drive')

# Set the working directory
%cd '/content/drive/MyDrive/RE/llama3RE'

# # For Hugging Face Hub setting
# from huggingface_hub import login

# # Upload the HuggingFace token (should have WRITE access) from Colab secrets
# HF = userdata.get('HF')

# # This is needed to upload the model to HuggingFace
# login(token=HF,add_to_git_credential=True)

# Create a path variable for the data folder
data_path = '/content/drive/MyDrive/RE/llama3RE/datas/'

# Full fine-tuning dataset
sft_dataset_file = f'{data_path}sft_train_data.json'

# Data collected from the the mini-test
mini_data_path = f'{data_path}mini_data.json'

# Test data containing all three outputs
all_tests_data = f'{data_path}all_tests.json'

# The adjusted training dataset
train_data_path = f'{data_path}sft_train_data.json'

# Create a path variable for the SFT model to be saved locally
sft_model_path = '/content/drive/MyDrive/RE/llama3RE/Llama3_RE/'

Mounted at /content/drive
/content/drive/MyDrive/RE/llama3RE


In [ ]:
# For Hugging Face Hub setting
from huggingface_hub import login

# Upload the HuggingFace token (should have WRITE access) from Colab secrets
HF = userdata.get('HF')

# This is needed to upload the model to HuggingFace
login(token=HF,add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("databricks/databricks-dolly-15k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [ ]:
# Choose the desired category from the dataset
ie_category = [e for e in dataset["train"] if e["category"]=="information_extraction"]

# Retain only the context from each instance
ie_context = [e["context"] for e in ie_category]

# Split the text into sentences (at the period) and keep the first sentence
reduced_context = [text.split('.')[0] + '.' for text in ie_context]

# Retain sequences of specified lengths only (use character length)
sampler = [e for e in reduced_context if 30 < len(e) < 170]

import json
hotpotqa_path = "/content/drive/MyDrive/RE/llama3RE/datas/hotpotqa.json"

# Load the HotpotQA JSON file
with open(hotpotqa_path, 'r') as file:
    hotpotqa_data = json.load(file)

# Extract questions
sampler = [item['question'] for item in hotpotqa_data]

sampler_length = len(sampler)
print(sampler_length)

7405


In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Used to securely store your API key
from google.colab import userdata

In [ ]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash


In [ ]:
# model = genai.GenerativeModel('gemini-1.5-flash')  # 모델 객체 생성 대신 모델 이름 저장
model_name = 'gemini'

# 이후 모델을 사용할 때는 다시 GenerativeModel 객체를 생성합니다.
model ='gemini'

System message 수정

In [ ]:
system_message = """
You are an experienced Named Entity Recognizer.
Your task is to extract entities from the given text and classify them as either 'core' or 'difference'.
- 'core' entities are the main subjects or topics that are consistent across variations of the query.
- 'difference' entities are the specific attributes or variations that distinguish different instances of the query.

The reason for extracting 'core' and 'difference' entities is to facilitate query decomposition. By recognizing 'core' and 'difference' entities, the query can be divided into multiple sub-queries for more precise information retrieval. For example, the query "Who was the Super Bowl MVP in 1979 and 1980?" can be decomposed into "Who was the Super Bowl MVP in 1979?" and "Who was the Super Bowl MVP in 1980?" based on the identified entities: Super Bowl MVP (core), 1979 (difference), and 1980 (difference).

Furthermore, the ultimate goal is to extract entities from the query, use them to create nodes, and build a knowledge graph. This knowledge graph will enable more efficient and accurate information retrieval and analysis.

I need to extract as much entity as possible because it is a ner for the purpose of graphing the query.

For each entity, include the entity name, its position in the sentence, and its tag (either core or difference).
Write the entities in the following format: EntityName (tag).
Make sure to recognize questions such as who and which as an entity. Except for PER, LOC, and TIME, you can name them as CONCEPT.

In the <Entities> area, put names for PER, LOC, ORG, TIME, CONCEPT
In the <Key points> area, put a name for core and difference.


====Two common-element-based questions====
Example:
Query: Who was the Super Bowl MVP in 1979 and 1980?
You:
Entities: Super Bowl MVP (CONCEPT), 1979 (TIME), 1980 (TIME)
Key Points: Super Bowl MVP (core), 1979 (difference), 1980 (difference)

Example:
Query: Are Chris Marker and Yvonne Rainer American?
You:
Entities: Chris Marker (PER), Yvonne Rainer (PER), American (CONCEPT)
Key Points: Chris Marker (difference), Yvonne Rainer (difference), American (core)

Example:
Query: How has Elon Musk's acquisition of X (formerly Twitter) impacted the stock prices and market valuation of related companies such as Tesla and SpaceX?
You:
Entities: Elon Musk's acquisition (CONCEPT), X (formerly Twitter) (ORG), stock prices (CONCEPT), market valuation (CONCEPT), Tesla (ORG), SpaceX (ORG)
Key Points: Elon Musk's acquisition (core), X (formerly Twitter) (core), stock prices (difference), market valuation (difference), Tesla (difference), SpaceX (difference)


====Two questions listed in parallel without common elements====
Example:
Query: What are the US policy toward China and China's policy toward the US?
You:
Entities: US policy (CONCEPT), China (LOC), China's policy (CONCEPT), US (LOC)
Key Points: US policy (difference), China (difference), China's policy (difference), US (difference)


Treat possessive forms (e.g., Mary's) as part of the same entity.
Do not add anything else.

Query :
"""

In [ ]:

model = genai.GenerativeModel('gemini-1.5-flash')
chat = model.start_chat(history=[])
response = chat.send_message(system_message)
to_markdown(response.text)

> Please provide me with the query you want me to analyze. I'm ready to extract entities and classify them as 'core' or 'difference'! 


In [ ]:
messages = [[
    {"role": "system","parts": f"{system_message}"},
    {"role": "user", "parts": e}] for e in sampler]


messages

[[{'role': 'system',
   'parts': '\nYou are an experienced Named Entity Recognizer.\nYour task is to extract entities from the given text and classify them as either \'core\' or \'difference\'.\n- \'core\' entities are the main subjects or topics that are consistent across variations of the query.\n- \'difference\' entities are the specific attributes or variations that distinguish different instances of the query.\n\nThe reason for extracting \'core\' and \'difference\' entities is to facilitate query decomposition. By recognizing \'core\' and \'difference\' entities, the query can be divided into multiple sub-queries for more precise information retrieval. For example, the query "Who was the Super Bowl MVP in 1979 and 1980?" can be decomposed into "Who was the Super Bowl MVP in 1979?" and "Who was the Super Bowl MVP in 1980?" based on the identified entities: Super Bowl MVP (core), 1979 (difference), and 1980 (difference).\n\nFurthermore, the ultimate goal is to extract entities from

In [ ]:
import json
import requests
from sklearn.metrics import precision_recall_fscore_support

# 테스트 데이터 로드
data = [
    {
        "text": "Were Scott Derrickson and Ed Wood of the same nationality?",
        "gold_ner": "Entities: Were (CONCEPT), Scott Derrickson (PER), Ed Wood (PER), same nationality (CONCEPT)\nKey Points: Were (core), Scott Derrickson (difference), Ed Wood (difference), same nationality (core)"
    }
]

In [ ]:
system_message = """
You are an experienced Named Entity Recognizer.
Your task is to extract entities from the given text and classify them as either 'core' or 'difference'.
- 'core' entities are the main subjects or topics that are consistent across variations of the query.
- 'difference' entities are the specific attributes or variations that distinguish different instances of the query.

The reason for extracting 'core' and 'difference' entities is to facilitate query decomposition. By recognizing 'core' and 'difference' entities, the query can be divided into multiple sub-queries for more precise information retrieval. For example, the query "Who was the Super Bowl MVP in 1979 and 1980?" can be decomposed into "Who was the Super Bowl MVP in 1979?" and "Who was the Super Bowl MVP in 1980?" based on the identified entities: Super Bowl MVP (core), 1979 (difference), and 1980 (difference).

Furthermore, the ultimate goal is to extract entities from the query, use them to create nodes, and build a knowledge graph. This knowledge graph will enable more efficient and accurate information retrieval and analysis.

I need to extract as much entity as possible because it is a ner for the purpose of graphing the query.

For each entity, include the entity name, its position in the sentence, and its tag (either core or difference).
Write the entities in the following format: EntityName (tag).
Make sure to recognize questions such as who and which as an entity. Except for PER, LOC, and TIME, you can name them as CONCEPT.

In the <Entities> area, put names for PER, LOC, ORG, TIME, CONCEPT
In the <Key points> area, put a name for core and difference.



Example:
Query: Who was the Super Bowl MVP in 1979 and 1980?
You:
Entities: Super Bowl MVP (CONCEPT), 1979 (TIME), 1980 (TIME)
Key Points: Super Bowl MVP (core), 1979 (difference), 1980 (difference)

Example:
Query: Are Chris Marker and Yvonne Rainer American?
You:
Entities: Chris Marker (PER), Yvonne Rainer (PER), American (CONCEPT)
Key Points: Chris Marker (difference), Yvonne Rainer (difference), American (core)

Example:
Query: How has Elon Musk's acquisition of X (formerly Twitter) impacted the stock prices and market valuation of related companies such as Tesla and SpaceX?
You:
Entities: Elon Musk's acquisition (CONCEPT), X (formerly Twitter) (ORG), stock prices (CONCEPT), market valuation (CONCEPT), Tesla (ORG), SpaceX (ORG)
Key Points: Elon Musk's acquisition (core), X (formerly Twitter) (core), stock prices (difference), market valuation (difference), Tesla (difference), SpaceX (difference)

Example:
Query: Who directed The Godfather and Apocalypse Now?
You:
Entities: Who (CONCEPT), The Godfather (CONCEPT), Apocalypse Now (CONCEPT)
Key Points: Who (core), The Godfather (difference), Apocalypse Now (difference)

Treat possessive forms (e.g., Mary's) as part of the same entity.
Do not add anything else.

Query :
"""

In [ ]:
# OpenAI Gemini model
#from openai import generative_models as genai
model = genai.GenerativeModel('gemini-1.5-flash')
chat = model.start_chat(history=[])

def get_model_response(text):
    response = chat.send_message(system_message + text)
    return response.text

def extract_entities(response):
    entities = []
    key_points = []
    lines = response.split('\n')
    for line in lines:
        if line.startswith("Entities:"):
            entities = line.replace("Entities:", "").strip().split(", ")
        elif line.startswith("Key Points:"):
            key_points = line.replace("Key Points:", "").strip().split(", ")
    return entities, key_points

gold_entities = []
pred_entities = []
gold_key_points = []
pred_key_points = []

for item in data:
    text = item['text']
    gold_ner = item['gold_ner']

    model_response = get_model_response(text)
    pred_entities_list, pred_key_points_list = extract_entities(model_response)

    gold_entities_list, gold_key_points_list = extract_entities(gold_ner)

    gold_entities.extend(gold_entities_list)
    pred_entities.extend(pred_entities_list)
    gold_key_points.extend(gold_key_points_list)
    pred_key_points.extend(pred_key_points_list)

# 정밀도, 재현율, F1 스코어 계산
entities_precision, entities_recall, entities_f1, _ = precision_recall_fscore_support(gold_entities, pred_entities, average='macro')
key_points_precision, key_points_recall, key_points_f1, _ = precision_recall_fscore_support(gold_key_points, pred_key_points, average='macro')

print(f"Entities - Precision: {entities_precision}, Recall: {entities_recall}, F1: {entities_f1}")
print(f"Key Points - Precision: {key_points_precision}, Recall: {key_points_recall}, F1: {key_points_f1}")

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

# **Claude**

In [ ]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
import anthropic
from google.colab import userdata
from sklearn.metrics import precision_recall_fscore_support

# Anthropics API 설정
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
)


In [ ]:
system_message = """
You are an experienced Named Entity Recognizer.
Your task is to extract entities from the given text and classify them as either 'core' or 'etc'.


The reason for extracting 'core' and 'etc' entities is to facilitate query decomposition. By recognizing 'core' and 'etc' entities, the query can be divided into multiple sub-queries for more precise information retrieval. For example, the query "Who was the Super Bowl MVP in 1979 and 1980?" can be decomposed into "Who was the Super Bowl MVP in 1979?" and "Who was the Super Bowl MVP in 1980?" based on the identified entities: Super Bowl MVP (core), 1979 (etc), and 1980 (etc).

- 'core' entities are the main subjects or topics that are consistent across variations of the query.The query will be divided based on this entity.
- 'etc' entities represent others except for 'core'

We're going to divide the query based on 'core',
so it's a condition that one entity has to have a relationship with two or more entities to become 'core'.

Furthermore, the ultimate goal is to extract entities from the query, use them to create nodes, and build a knowledge graph. This knowledge graph will enable more efficient and accurate information retrieval and analysis.
If it is judged that there is no 'core', you can judge all entities as 'etc'.


I'll provide the query, the entities, and the relationship between them.
Write the entities in the following format: EntityName (tag).



Example:
Query: Who was the Super Bowl MVP in 1979 and 1980?\nEntity: Super Bowl MVP, 1979, 1980\nSuper Bowl MVP|was|1979\nSuper Bowl MVP|was|1980
You:
Super Bowl MVP (core), 1979 (etc), 1980 (etc)

Example:
Query: Are Chris Marker and Yvonne Rainer American?\nEntity: Chris Marker, Yvonne Rainer, American\nChris Marker|is|American\nYvonne Rainer|is|American
You:
Chris Marker (etc), Yvonne Rainer (etc), American (core)

Example:
Query: How has Elon Musk's acquisition of X (formerly Twitter) impacted the stock prices and market valuation of related companies such as Tesla and SpaceX?\nEntity: Elon Musk's acquisition, X (formerly Twitter), stock prices, market valuation, Tesla , SpaceX\nElon Musk's acquisition|impacts|stock prices\nElon Musk's acquisition|impacts|market valuation\nElon Musk's acquisition|impacts|Tesla\nElon Musk's acquisition|impacts|SpaceX
You:
Elon Musk's acquisition (etc), X (formerly Twitter) (etc), stock prices (etc), market valuation of related companies (core), Tesla (etc), SpaceX (etc)

Example:
Query: Who directed The Godfather and Apocalypse Now?\nEntity: Who, The Godfather, Apocalypse Now\nWho|directed|The Godfather\nWho|directed|Apocalypse Now
You:
Who (core), The Godfather (etc), Apocalypse Now (etc)

Treat possessive forms (e.g., Mary's) as part of the same entity.
Do not add anything else.

Keep in mind that you can't be a 'core' if you don't get that entity more than once in the relationship I give you
"""

In [ ]:
import anthropic #삭제할 섹션
from google.colab import userdata

ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.0,
    system=system_message,
    messages=[
        {"role": "user", "content": "Were Scott Derrickson and Ed Wood of the same nationality?"}
    ]
)

print(message.content[0].text)

Scott Derrickson (etc), Ed Wood (etc), nationality (core)


In [ ]:
import json
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
def get_model_response(query):
    system_message = """
    You are an experienced Named Entity Recognizer.
    Your task is to extract entities from the given text and classify them as either 'core' or 'etc'.


    The reason for extracting 'core' and 'etc' entities is to facilitate query decomposition. By recognizing 'core' and 'etc' entities, the query can be divided into multiple sub-queries for more precise information retrieval. For example, the query "Who was the Super Bowl MVP in 1979 and 1980?" can be decomposed into "Who was the Super Bowl MVP in 1979?" and "Who was the Super Bowl MVP in 1980?" based on the identified entities: Super Bowl MVP (core), 1979 (etc), and 1980 (etc).

    - 'core' entities are the main subjects or topics that are consistent across variations of the query.The query will be divided based on this entity.
    - 'etc' entities represent others except for 'core'

    We're going to divide the query based on 'core',
    so it's a condition that one entity has to have a relationship with two or more entities to become 'core'.

    Furthermore, the ultimate goal is to extract entities from the query, use them to create nodes, and build a knowledge graph. This knowledge graph will enable more efficient and accurate information retrieval and analysis.
    If it is judged that there is no 'core', you can judge all entities as 'etc'.


    I'll provide the query, the entities, and the relationship between them.
    Write the entities in the following format: EntityName (tag).



    Example:
    Query: Who was the Super Bowl MVP in 1979 and 1980?\nEntity: Super Bowl MVP, 1979, 1980\nSuper Bowl MVP|was|1979\nSuper Bowl MVP|was|1980
    You:
    Super Bowl MVP (core), 1979 (etc), 1980 (etc)

    Example:
    Query: Are Chris Marker and Yvonne Rainer American?\nEntity: Chris Marker, Yvonne Rainer, American\nChris Marker|is|American\nYvonne Rainer|is|American
    You:
    Chris Marker (etc), Yvonne Rainer (etc), American (core)

    Example:
    Query: How has Elon Musk's acquisition of X (formerly Twitter) impacted the stock prices and market valuation of related companies such as Tesla and SpaceX?\nEntity: Elon Musk's acquisition, X (formerly Twitter), stock prices, market valuation, Tesla , SpaceX\nElon Musk's acquisition|impacts|stock prices\nElon Musk's acquisition|impacts|market valuation\nElon Musk's acquisition|impacts|Tesla\nElon Musk's acquisition|impacts|SpaceX
    You:
    Elon Musk's acquisition (etc), X (formerly Twitter) (etc), stock prices (etc), market valuation of related companies (core), Tesla (etc), SpaceX (etc)

    Example:
    Query: Who directed The Godfather and Apocalypse Now?\nEntity: Who, The Godfather, Apocalypse Now\nWho|directed|The Godfather\nWho|directed|Apocalypse Now
    You:
    Who (core), The Godfather (etc), Apocalypse Now (etc)

    Treat possessive forms (e.g., Mary's) as part of the same entity.
    Do not add anything else.
    When you present the results, you must present the results in the form of all the entities I gave you. Don't get rid of the entity
    Keep in mind that you can't be a 'core' if you don't get that entity more than once in the relationship I give you

    """
    #system_message = system_message + query
    message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.0,
    system=system_message,
    messages=[
        {"role": "user", "content": query}
    ]
)
    return message.content[0].text

def extract_entities(ner_output):
    # Parse the output to extract entities and their tags
    entities = []
    if ',' in ner_output:
        entities_str = ner_output.split('\n')[-1].strip()
        entities = [tuple(e.strip().split(" ")) for e in entities_str.split(", ")]
    return entities

def get_entities_and_keypoints(query):
    response = get_model_response(query)
    return response

def evaluate_model(test_data):
    y_true = []
    y_pred = []
    incorrect_predictions = []

    for item in test_data:
        query = item["input"]
        gold_ner = item["output"]

        predicted_ner = get_entities_and_keypoints(query)

        gold_entities = extract_entities(gold_ner)
        predicted_entities = extract_entities(predicted_ner)

        y_true.append(set(gold_entities))
        y_pred.append(set(predicted_entities))

        # Check if prediction is incorrect
        if set(gold_entities) != set(predicted_entities):
            incorrect_predictions.append({
                "query": query,
                "gold_entities": gold_entities,
                "predicted_entities": predicted_entities
            })

    mlb = MultiLabelBinarizer()
    y_true_bin = mlb.fit_transform(y_true)
    y_pred_bin = mlb.transform(y_pred)

    f1 = f1_score(y_true_bin, y_pred_bin, average='weighted')

    return f1, incorrect_predictions
# Assuming `data` is loaded from the JSON file
with open('/content/output_valid.json', 'r') as file:
    data = json.load(file)

# Evaluate the model and get incorrect predictions
f1, incorrect_predictions = evaluate_model(data)
print(f"F1 Score: {f1}")

# Output incorrect predictions
for incorrect in incorrect_predictions:
    print("Query:", incorrect["query"])
    print("Gold Entities:", incorrect["gold_entities"])
    print("Predicted Entities:", incorrect["predicted_entities"])
    print()

F1 Score: 0.9080117354998871
Query: Query: What is the inhabitant of the city where  122nd SS-Standarte was formed in2014
Entity: inhabitant, What, city, 2014, 122nd SS-Standarte
What|is|inhabitant
inhabitant|of|city
122nd SS-Standarte|formed in|city
122nd SS-Standarte|formed in|2014
Gold Entities: [('inhabitant', '(etc)'), ('What', '(etc)'), ('city', '(core)'), ('2014', '(etc)'), ('122nd', 'SS-Standarte', '(etc)')]
Predicted Entities: [('inhabitant', '(core)'), ('What', '(etc)'), ('city', '(etc)'), ('2014', '(etc)'), ('122nd', 'SS-Standarte', '(etc)')]

Query: Query: Alvaro Mexia had a diplomatic mission with which tribe of indigenous people?
Entity: Alvaro Mexia, which tribe, diplomatic mission, indigenous people
Alvaro Mexia|had|diplomatic mission
diplomatic mission|with|which tribe
which tribe|of|indigenous people
Gold Entities: [('Alvaro', 'Mexia', '(etc)'), ('which', 'tribe', '(etc)'), ('diplomatic', 'mission', '(etc)'), ('indigenous', 'people', '(etc)')]
Predicted Entities: [('A

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [('18th', 'Independent', 'Spirit', 'Awards', '(core)'), ('Major', '(core)'), ('Nina', '(etc)'), ('U.S.', '(core)'), ('airplane', '(etc)'), ('and', 'beverage', 'company', '(etc)'), ('annual', 'documentary', 'film', 'festival', '(etc)'), ('bands', '(etc)'), ('brother', '(etc)'), ('city', '(etc)'), ('descendants', '(etc)'), ('diplomatic', 'mission', '(core)'), ('first', 'round', 'draft', 'pick', '(core)'), ('inhabitant', '(core)'), ('month', '(core)'), ('novel', '(core)'), ('song', '(etc)')] will be ignored
  warnings.warn(


In [ ]:
import anthropic
from google.colab import userdata
NER_prompt = """
You are a capable entity extractor.
You need to extract all Entities from the given sentence (query, question).
You need to extract all the entities that have meaning in the sentence.
All Entities should be extracted in the form of Entities, Entities, Entities.
When extracting entity, extract as much as possible.
interrogative word must should be treated as an entity Where was it located.
Don't print anything other than what you asked

e.g. )
Question: What measures might the international community take if X (formerly Twitter) fails to comply with the European Union's Code?

->
What, measures, international community, X (formerly Twitter), European Union's Code

e.g. )
Question: Who was the Super Bowl MVP in 1979 and 1980.

->
Who, Super Bowl MVP, 1979 and 1980

e.g. )
Question: 2014 S/S is the debut album of a South Korean boy group that was formed by who?

->
2014 S/S, debut album, South Korean boy group, who
"""
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.0,
    system=NER_prompt,
    messages=[
        {"role": "user", "content": "query"}
    ]
)

print(message.content[0].text)